#**Rapid classification of amorphous-crystalline phase using artificial neural networks and resistivity mapping Ⅰ**

This script is to reproduce the models described in the article entitled of "Electrical Resistivity as a Descriptor for Classification of Amorphous-Crystalline Phases of Alloys," authored by Daegun You, Haitao Zhang, Shraddha Ganorkar, Taeyeop Kim, Jan Schroers, Joost J. Vlassak, and Dongwoo Lee. This is exported version for web-based running, so some minor discrepancies occur with compared to the original script (refer to the original page on github). First, you should download the matrix data file named of 'ML-R-MG.mat' in the original page. Then, it will ask you to upload the matrix again in this page.

In [1]:
!pip install mat4py
import numpy
import numpy as np
from mat4py import loadmat
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam

def scaler(X):
    X = (X-np.mean(X, axis=0))/np.std(X, axis=0, ddof=1)
    return X

from google.colab import files
file_uploaded = files.upload()

Saving ML-R-MG.mat to ML-R-MG.mat


Load training/test datasets as matrix that includes all features and output (already divided into two sets randomly 20% selected test data) Literature data is also loaded for independent test (transferability test to the multicomponent systems and scales)
###### 0-absolute resistivity, 1-modified resistivity
###### 2-average atomic radii, 3-atomic mismatch
###### 4-heat of mixing, 5-average electronegativity
###### 6-electronegativity difference, 7-number of atom
###### 8-Amorphous/Crystalline label (1/0)

In [2]:
data = loadmat('ML-R-MG.mat')
data_training = data['trainingset_HTEonly']
data_test = data['testset_HTEonly']
data_liter = data['allset_Literature']
data_training = np.asarray(data_training, dtype=np.float64)
data_test = np.asarray(data_test, dtype=np.float64)
data_liter = np.asarray(data_liter, dtype=np.float64)

Scaling input features on the total domain (training+test)
\
(literature data would be scaled within the known scaling domain)

In [3]:
X1 = scaler(np.vstack((data_training[:, :8], data_test[:, :8])))
X2 = scaler(np.vstack((data_training[:, :8], data_test[:, :8], data_liter[:, :8])))

Define training and test input, and their output as well.\
Input example below can be varied on the specific utilization of features as you want

In [4]:
x_train = np.c_[X1[:242,1],X1[:242,3],X1[:242,5]]
x_test = np.c_[X1[242:,1],X1[242:,3],X1[242:,5]]
x_ind = np.c_[X2[302:,1],X2[302:,3],X2[302:,5]]
y_train = data_training[:, 8]
y_test = data_test[:, 8]
y_ind = data_liter[:, 8]

Reshaping dataset as for network flow

In [5]:
x_train = x_train.reshape(242, 3)
x_test = x_test.reshape(60, 3)
x_ind = x_ind.reshape(406, 3)
y_train = y_train.reshape(242,1)
y_test = y_test.reshape(60,1)
y_ind = y_ind.reshape(406, 1)

Early stopping the model training by validation loss

In [6]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

Set learning rate in Adam optimizer (this value has been already optimized)

In [7]:
opt = Adam(lr=1e-2)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Artificial neural networks of 50 neurons on 1-hidden layer (this has been already optimized)

In [8]:
mdl = Sequential()
mdl.add(Dense(50, input_dim=3, activation='relu'))
mdl.add(Dense(1, activation='sigmoid'))
mdl.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])
hst = mdl.fit(x_train, y_train, batch_size=20, epochs=1500,
            validation_data=(x_test, y_test), callbacks=[es, mc])

Epoch 1/1500
13/13 [==============================] - 1s 17ms/step - loss: 0.5928 - accuracy: 0.6157 - val_loss: 0.4642 - val_accuracy: 0.8000
Epoch 2/1500
13/13 [==============================] - 0s 3ms/step - loss: 0.4752 - accuracy: 0.7810 - val_loss: 0.4020 - val_accuracy: 0.8667
Epoch 3/1500
13/13 [==============================] - 0s 3ms/step - loss: 0.4424 - accuracy: 0.7851 - val_loss: 0.3971 - val_accuracy: 0.8500
Epoch 4/1500
13/13 [==============================] - 0s 3ms/step - loss: 0.4208 - accuracy: 0.8099 - val_loss: 0.3678 - val_accuracy: 0.9167
Epoch 5/1500
13/13 [==============================] - 0s 3ms/step - loss: 0.4099 - accuracy: 0.8017 - val_loss: 0.3465 - val_accuracy: 0.9167
Epoch 6/1500
13/13 [==============================] - 0s 3ms/step - loss: 0.3967 - accuracy: 0.8058 - val_loss: 0.3419 - val_accuracy: 0.9167
Epoch 7/1500
13/13 [==============================] - 0s 3ms/step - loss: 0.3899 - accuracy: 0.8058 - val_loss: 0.3285 - val_accuracy: 0.9167
Epoch

Test accuracy

In [9]:
print("\r Test accuracy : %.4f" % (mdl.evaluate(x_test, y_test)[1])*100)

2/2 [==============================] - 0s 7ms/step - loss: 0.3834 - accuracy: 0.8833
 Test accuracy : 0.8833


Test accuracy of literature data for transferability

In [10]:
print("\r Accuracy on literature : %.4f" % (mdl.evaluate(x_ind, y_ind)[1])*100)

13/13 [==============================] - 0s 2ms/step - loss: 1.9946 - accuracy: 0.7488
 Accuracy on literature : 0.7488
